# MLP Gradient - pytorch

前向和反向计算量时一样的吗？

分析一层网络和多层网络计算量

## use pytorch auto grad

In [69]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

In [70]:
bs = 2
dim = 3
out_dim = 6
head = 4

In [73]:
x = torch.randn(bs, dim)
w1 = torch.randn(dim, out_dim, requires_grad=True)
w2 = torch.randn(out_dim, head)
y_label = torch.randn(bs, head)

## forward

In [77]:
y1 = x @ w1
y1.retain_grad()
y2 = y1 @ w2
y2.retain_grad()
loss_mse = ((y_label - y2) ** 2)
loss_mse.retain_grad()
loss = loss_mse.mean()

## backward

In [79]:
loss.backward()
print(loss)

tensor(25.9975, grad_fn=<MeanBackward0>)

### mlp-layer2

In [102]:
# pytorch求导结果
# print(y2.grad)
print(w2.grad)
print(y1.grad)

tensor([[-2.4221,  1.1051, -4.8241, -4.0603],
        [-4.0850, -0.5590, -5.4524, -3.1321],
        [ 1.1359, -1.3301,  3.1616,  3.1493],
        [ 0.4604,  3.0697, -2.7160, -4.2585],
        [-0.7690,  0.9133, -2.1546, -2.1518],
        [-0.7214, -2.3768,  1.5605,  2.9408]])

tensor([[ 3.5725,  2.8872, -5.9663,  0.3279,  4.7426, -1.7715],
        [ 6.1835,  5.0278, -3.1847, -0.0744,  6.1692, -0.6577]])

In [103]:
# 手动计算
e = 2 * (y2-y_label) / (bs * head)
print(e)

w2_grad = y1.t() @ e
print(w2_grad)

y1_grad = e @ w2.t()
print(y1_grad)

tensor([[-4.5383e-02,  1.0484e+00, -1.2288e+00, -1.6523e+00],
        [-1.3571e+00, -1.1370e-03, -2.0159e+00, -1.3236e+00]],
       grad_fn=<DivBackward0>)

tensor([[-2.4221,  1.1051, -4.8241, -4.0603],
        [-4.0850, -0.5590, -5.4524, -3.1321],
        [ 1.1359, -1.3301,  3.1616,  3.1493],
        [ 0.4604,  3.0697, -2.7160, -4.2585],
        [-0.7690,  0.9133, -2.1546, -2.1518],
        [-0.7214, -2.3768,  1.5605,  2.9408]], grad_fn=<MmBackward0>)

tensor([[ 3.5725,  2.8872, -5.9663,  0.3279,  4.7426, -1.7715],
        [ 6.1835,  5.0278, -3.1847, -0.0744,  6.1692, -0.6577]],
       grad_fn=<MmBackward0>)

### mlp-layer1

In [106]:
# pytorch
print(w1.grad)
print(x.grad)

tensor([[12.0253,  9.7697, -8.1196,  0.0284, 12.5476, -1.9287],
        [-0.5551, -0.4459,  1.5576, -0.1076, -0.9170,  0.4879],
        [ 4.7383,  3.8829,  4.6353, -0.6931,  2.7066,  1.8821]])

tensor([[18.6402, -1.3831, -5.7146],
        [21.0441, -4.9296, -1.8759]])

In [108]:
# 手动求导
w1_grad = x.t() @ y1_grad
print(w1_grad)

x1_grad = y1_grad @ w1.t()
print(x1_grad)

tensor([[12.0253,  9.7697, -8.1196,  0.0284, 12.5476, -1.9287],
        [-0.5551, -0.4459,  1.5576, -0.1076, -0.9170,  0.4879],
        [ 4.7383,  3.8829,  4.6353, -0.6931,  2.7066,  1.8821]],
       grad_fn=<MmBackward0>)

tensor([[18.6402, -1.3831, -5.7146],
        [21.0441, -4.9296, -1.8759]], grad_fn=<MmBackward0>)

# 结论

梯度计算是前向计算的两倍计算量。

多层：2倍
单层：同样的计算量，因为y=wx, 对于x的导数不用计算